In [ ]:
#https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [1]:
import collections
import numpy as np


#TENSORFLOW
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [2]:
with open('fra-eng/fra.txt','r') as f:
    data = f.read()
    print('number of samples:',len(data.split('\n')))

number of samples: 178010


In [99]:
uncleaned_data_list = data.split('\n')
english_sentences = []
french_sentences = []

import random
sampled = random.sample(uncleaned_data_list,5000)
#sampled = uncleaned_data_list

for sentence in sampled:
    try:
        english_sentences.append(sentence.split('\t')[:-1][0])
        french_sentences.append(sentence.split('\t')[:-1][1])
    except:
        print(sentence)

In [100]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])


print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

30608 English words.
5038 unique English words.
10 Most common words in the English dataset:
"I" "to" "you" "the" "a" "is" "Tom" "of" "in" "that"

33427 French words.
6528 unique French words.
10 Most common words in the French dataset:
"de" "Je" "?" "pas" "que" "à" "ne" "la" "le" "Il"


In [101]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk


text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']


text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [102]:
def test_pad(pad):
    tokens = [
        [i for i in range(4)],
        [i for i in range(6)],
        [i for i in range(3)]]
    padded_tokens = pad(tokens)
    padding_id = padded_tokens[0][-1]
    true_padded_tokens = np.array([
        [i for i in range(4)] + [padding_id]*2,
        [i for i in range(6)],
        [i for i in range(3)] + [padding_id]*3])
    assert isinstance(padded_tokens, np.ndarray),\
        'Pad returned the wrong type.  Found {} type, expected numpy array type.'
    assert np.all(padded_tokens == true_padded_tokens), 'Pad returned the wrong results.'

    padded_tokens_using_length = pad(tokens, 9)
    assert np.all(padded_tokens_using_length == np.concatenate((true_padded_tokens, np.full((3, 3), padding_id)), axis=1)),\
        'Using length argument return incorrect results'

In [103]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')
test_pad(pad)
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [104]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk


preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)


print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 44
Max French sentence length: 55
English vocabulary size: 3403
French vocabulary size: 5151


In [105]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print('logits_to_text() function loaded.')

logits_to_text() function loaded.


In [154]:
import matplotlib.pyplot as plt
import numpy
# list all data in history
def plot_history(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [106]:
def simple_model(input_shape, output_sequence_length, output_vocab_size):
    
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(output_vocab_size+1))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

#tests.test_simple_model(simple_model)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# Train the neural network


simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    french_vocab_size+1)

#simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=10, epochs=10, validation_split=0.2)
history = simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=10, epochs=25, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/25
400/400 [==============================] - 22s 56ms/step - loss: 1.7314 - accuracy: 0.8759 - val_loss: 0.8793 - val_accuracy: 0.8784
Epoch 2/25
400/400 [==============================] - 22s 55ms/step - loss: 0.8584 - accuracy: 0.8791 - val_loss: 0.8719 - val_accuracy: 0.8816
Epoch 3/25
400/400 [==============================] - 22s 55ms/step - loss: 0.8405 - accuracy: 0.8821 - val_loss: 0.8665 - val_accuracy: 0.8824
Epoch 4/25
400/400 [==============================] - 23s 58ms/step - loss: 0.8245 - accuracy: 0.8825 - val_loss: 0.8601 - val_accuracy: 0.8835
Epoch 5/25
400/400 [==============================] - 23s 59ms/step - loss: 0.8097 - accuracy: 0.8832 - val_loss: 0.8537 - val_accuracy: 0.8844
Epoch 6/25
400/400 [==============================] - 22s 56ms/step - loss: 0.7959 - accuracy: 0.8845 - val_loss: 0.8487 - val_accuracy: 0.8852
Epoch 7/25
400/400 [==============================] - 22s 55ms/step - loss: 0.7835 - accuracy: 0.8856 - val_loss: 0.8465 - val_accuracy:

In [125]:
from keras.models import Sequential
from keras.layers import Embedding
def embed_model(input_shape, output_sequence_length, french_vocab_size):
    
    learning_rate = 1e-3
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(Embedding(french_vocab_size, 64, input_length=input_shape[1]))
    model.add(GRU(64, return_sequences=True, activation="tanh"))
    model.add(TimeDistributed(Dense(french_vocab_size, activation="softmax")))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    french_vocab_size+1)


history = embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=10, epochs=25, validation_split=0.2)
plot_history(history)

Epoch 1/25
400/400 [==============================] - 16s 41ms/step - loss: 1.6871 - accuracy: 0.8741 - val_loss: 0.9504 - val_accuracy: 0.8784
Epoch 2/25
400/400 [==============================] - 16s 40ms/step - loss: 0.8808 - accuracy: 0.8807 - val_loss: 0.8552 - val_accuracy: 0.8823
Epoch 3/25
400/400 [==============================] - 16s 40ms/step - loss: 0.8211 - accuracy: 0.8830 - val_loss: 0.8304 - val_accuracy: 0.8840
Epoch 4/25
400/400 [==============================] - 16s 39ms/step - loss: 0.7911 - accuracy: 0.8860 - val_loss: 0.8173 - val_accuracy: 0.8878
Epoch 5/25
400/400 [==============================] - 16s 39ms/step - loss: 0.7631 - accuracy: 0.8896 - val_loss: 0.8055 - val_accuracy: 0.8903
Epoch 6/25
400/400 [==============================] - 15s 39ms/step - loss: 0.7344 - accuracy: 0.8923 - val_loss: 0.7910 - val_accuracy: 0.8925
Epoch 7/25
400/400 [==============================] - 16s 40ms/step - loss: 0.7058 - accuracy: 0.8949 - val_loss: 0.7827 - val_accuracy:

In [135]:
def bd_model(input_shape, output_sequence_length, french_vocab_size):
   
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model


tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(french_tokenizer.word_index)+1)


history = bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=10, epochs=25, validation_split=0.2)
plot_history(history)

Epoch 1/25
400/400 [==============================] - 21s 51ms/step - loss: 1.2547 - accuracy: 0.8807 - val_loss: 0.8512 - val_accuracy: 0.8839
Epoch 2/25
400/400 [==============================] - 21s 52ms/step - loss: 0.8153 - accuracy: 0.8845 - val_loss: 0.8541 - val_accuracy: 0.8853
Epoch 3/25
400/400 [==============================] - 21s 52ms/step - loss: 0.7926 - accuracy: 0.8856 - val_loss: 0.8481 - val_accuracy: 0.8867
Epoch 4/25
400/400 [==============================] - 21s 53ms/step - loss: 0.7729 - accuracy: 0.8867 - val_loss: 0.8465 - val_accuracy: 0.8870
Epoch 5/25
400/400 [==============================] - 22s 54ms/step - loss: 0.7557 - accuracy: 0.8875 - val_loss: 0.8467 - val_accuracy: 0.8881
Epoch 6/25
400/400 [==============================] - 21s 54ms/step - loss: 0.7380 - accuracy: 0.8881 - val_loss: 0.8438 - val_accuracy: 0.8891
Epoch 7/25
400/400 [==============================] - 21s 52ms/step - loss: 0.7191 - accuracy: 0.8892 - val_loss: 0.8420 - val_accuracy:

In [152]:
def encdec_model(input_shape, output_sequence_length, french_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(french_tokenizer.word_index)+1)

history = encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=10, epochs=25, validation_split=0.2)
plot_history(history)

Epoch 1/25
400/400 [==============================] - 22s 55ms/step - loss: 1.2840 - accuracy: 0.8801 - val_loss: 0.8998 - val_accuracy: 0.8819
Epoch 2/25
400/400 [==============================] - 21s 52ms/step - loss: 0.8781 - accuracy: 0.8821 - val_loss: 0.8948 - val_accuracy: 0.8819
Epoch 3/25
400/400 [==============================] - 21s 53ms/step - loss: 0.8651 - accuracy: 0.8823 - val_loss: 0.9001 - val_accuracy: 0.8826
Epoch 4/25
400/400 [==============================] - 21s 52ms/step - loss: 0.8564 - accuracy: 0.8834 - val_loss: 0.8974 - val_accuracy: 0.8833
Epoch 5/25
400/400 [==============================] - 20s 50ms/step - loss: 0.8489 - accuracy: 0.8833 - val_loss: 0.8975 - val_accuracy: 0.8837
Epoch 6/25
400/400 [==============================] - 20s 50ms/step - loss: 0.8436 - accuracy: 0.8838 - val_loss: 0.9006 - val_accuracy: 0.8837
Epoch 7/25
400/400 [==============================] - 21s 51ms/step - loss: 0.8408 - accuracy: 0.8839 - val_loss: 0.9138 - val_accuracy:

In [171]:
def custom_model(input_shape, output_sequence_length, french_vocab_size):
  
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model


tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
custosm = model_custom(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(french_tokenizer.word_index)+1)


history = custosm.fit(tmp_x, preproc_french_sentences, batch_size = 10, epochs = 25, validation_split = 0.2)
plot_history(history)

Epoch 1/25
400/400 [==============================] - ETA: 0s - loss: 1.1098 - accuracy: 0.8792

InvalidArgumentError:  indices[6,6] = 3403 is not in [0, 3403)
	 [[node sequential_14/embedding_5/embedding_lookup (defined at <ipython-input-171-7dc2bef198c0>:26) ]] [Op:__inference_test_function_252710]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_14/embedding_5/embedding_lookup:
 sequential_14/embedding_5/embedding_lookup/251065 (defined at /opt/anaconda3/lib/python3.6/contextlib.py:81)

Function call stack:
test_function


In [160]:
def translate(x, y, x_tk, y_tk, model, sentence):
    tmp_X = pad(preproc_english_sentences)
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    
    predictions = model.predict(sentence)
    output = [y_id_to_word[np.argmax(x)] for x in predictions[0]]
    
    translation = []
    for word in output:
        if word == '<PAD>':
            break
        else:
            translation.append(word)
    print(' '.join(translation))

sentence = 'where am i from'
#m = simple_rnn_model
#m = embeded_model
m = bidi_model
#m = encodeco_model
translate(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer, m, sentence)

ValueError: in user code:

    /opt/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /opt/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1445 run_step 